<h1>Part 1</h1>

In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pymongo

In [3]:
def scrape_nasa_news(url,extra_posts=False):
    '''Use this function to scrape the article titles and descriptions from 
    the nasa mars homepage and save them to mongo. 
    '''
    # initialize mongo 
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)

    # initializing mongo db and collection 
    nasa_db = client.nasa_db
    nasa_collection = nasa_db.articles

    # declaring element names to grab
    article_header = 'slide'
    content_title = 'content_title'
    content_body = 'article_teaser_body'
    wait_element = 'news'
    
    # intializing webdriver
    # driver = webdriver.Chrome(executable_path='C:/path/to/chromedriver.exe')
    driver = webdriver.Chrome(executable_path='C:/Users/janeb/Documents/USC Data Analytics Bootcamp/11-Web-Scraping-and-Document-Databases/Homework/chromedriver.exe')
    url = "https://mars.nasa.gov/news/"
    # get url and wait until page is fully loaded before proceeding
    driver.get(url)
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID,wait_element))
        )
        # Prompt user for amount of posts to be returned
        if extra_posts:
            while True:
                print('By default the site returns an 40 posts')
                print(f'How many extra posts i.e. 1 = 40, 2 = 80,3 = 120')
                posts = int(input())
                if posts > 0:
                    break
            # Grab a the more button element from the html
            more_button = driver.find_element_by_xpath('/html/body/div[3]/div/div[3]/div[3]/div/article/div/section/div/footer/a')
            # click the more button to retrieve more posts
            for post in range(posts):
                more_button.click()
                time.sleep(1)
            
        # Return a list of divs that hold the articles
        articles = driver.find_elements_by_class_name(article_header)
        # Iterate though and parse the articles to grab the titles and content
        for article in articles:
        # If element exists pass to a variable and create an object
            try:
                title = article.find_element_by_class_name(content_title).text
                body = article.find_element_by_class_name(content_body).text
                # post object with post title and body
                post = {
                    'title':title,
                    'body': body
                }
                #insert the post object into mongo
                nasa_collection.insert_one(post)
                print(post)
            except:
                continue
    except Exception as e:
        print(e)
    finally:
        driver.quit()

url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
scrape_nasa_news(url,extra_posts=True)

By default the site returns an 40 posts
How many extra posts i.e. 1 = 40, 2 = 80,3 = 120
1
{'title': "NASA's Curiosity Keeps Rolling As Team Operates Rover From Home", 'body': 'The team has learned to meet new challenges as they work remotely on the Mars mission.', '_id': ObjectId('5e9c9ae1197c7cf8beeb9eda')}
{'title': "Mars Helicopter Attached to NASA's Perseverance Rover", 'body': "The team also fueled the rover's sky crane to get ready for this summer's history-making launch.", '_id': ObjectId('5e9c9ae1197c7cf8beeb9edb')}
{'title': "NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes", 'body': 'After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.', '_id': ObjectId('5e9c9ae1197c7cf8beeb9edc')}
{'title': 'The Man Who Wanted to Fly on Mars', 'body': "The Mars Helicopter is riding to the Red Planet this summer with NASA's Perseverance rover. The helicopter's chief engineer, Bob Balar

{'title': 'Two of a Space Kind: Apollo 12 and Mars 2020', 'body': 'Apollo 12 and the upcoming Mars 2020 mission may be separated by half a century, but they share several goals unique in the annals of space exploration.', '_id': ObjectId('5e9c9ae2197c7cf8beeb9efb')}
{'title': 'Mars Scientists Investigate Ancient Life in Australia', 'body': "Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions.", '_id': ObjectId('5e9c9ae2197c7cf8beeb9efc')}
{'title': "NASA's Mars 2020 Will Hunt for Microscopic Fossils", 'body': "A new paper identifies a ring of minerals at the rover's landing site that are ideal for fossilizing microbial life.", '_id': ObjectId('5e9c9ae2197c7cf8beeb9efd')}
{'title': 'With Mars Methane Mystery Unsolved, Curiosity Serves Scientists a New One: Oxygen', 'body': 'For the first time in the history of space exploration, scientists have measured the seasonal changes i

{'title': 'Mars 2020 Rover: T-Minus One Year and Counting', 'body': "The launch period for NASA's next rover, Mars 2020, opens exactly one year from today, July 17, 2020, and extends through Aug. 5, 2020.", '_id': ObjectId('5e9c9ae4197c7cf8beeb9f1d')}
{'title': 'NASA Racks Up Two Emmy Nominations for Mission Coverage', 'body': "JPL's coverage of the Mars InSight mission is among the efforts that will be up for an award in mid-September.", '_id': ObjectId('5e9c9ae4197c7cf8beeb9f1e')}
{'title': 'Want to Colonize Mars? Aerogel Could Help', 'body': 'Researchers are studying whether a wonder material used in Mars rovers could help warm parts of the Red Planet rich in water ice.', '_id': ObjectId('5e9c9ae4197c7cf8beeb9f1f')}
{'title': 'A Rover Pit Stop at JPL', 'body': "Working like a finely honed machine, a team of engineers in this time-lapse video clip install test wheels on another finely honed machine: NASA's Mars 2020 rover.", '_id': ObjectId('5e9c9ae4197c7cf8beeb9f20')}
{'title': 'Mar

<h1>Part 2</h1>

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
featured_image_response = requests.get(featured_image_url)
if featured_image_response.status_code == 200:
    featured_image_html = featured_image_response.text

In [6]:
featured_image_soup = BeautifulSoup(featured_image_html, "html.parser")
image_element = featured_image_soup.find("a", {"class": "button fancybox"}).get('data-fancybox-href')
"https://www.jpl.nasa.gov" + image_element

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA15253_ip.jpg'

<h1>Part 3</h1>

In [10]:
weather_tweets_url = "https://twitter.com/marswxreport?lang=en"
driver = webdriver.Chrome(executable_path='C:/Users/janeb/Documents/USC Data Analytics Bootcamp/11-Web-Scraping-and-Document-Databases/Homework/chromedriver.exe')
driver.get(weather_tweets_url)

In [11]:
#Get using Xpath
mars_weather = driver.find_element_by_xpath("/html/body/div/div/div/div[2]/main/div/div/div/div/div/div/div/div/div[2]/section/div/div/div/div[1]/div/div/div/article/div/div[2]/div[2]/div[2]/div[1]/div/span")
mars_weather.text

'InSight sol 495 (2020-04-17) low -94.0ºC (-137.2ºF) high -4.2ºC (24.4ºF)\nwinds from the WNW at 4.6 m/s (10.4 mph) gusting to 16.7 m/s (37.3 mph)\npressure at 6.60 hPa'

<h1>Part 4</h1>

In [13]:
import pandas as pd

In [14]:
mars_fact_url = "https://space-facts.com/mars/"
all_mars_fact_tables = pd.read_html(mars_fact_url)
df_mars_facts = all_mars_fact_tables[0]
df_mars_facts.columns = ["Measure", "Number"]
df_mars_facts.head(10)

,Measure,Number
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


<h1>Part 5</h1>

In [16]:
mars_hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
mars_hemisphere_response = requests.get(mars_hemisphere_url)
if mars_hemisphere_response.status_code == 200:
    mars_hemisphere_html = mars_hemisphere_response.text

In [20]:
def get_full_image_link(link):
    response = requests.get(link)
    if response.status_code == 200:
        html = response.text
    soup = BeautifulSoup(html, "html.parser")
    img_link = soup.find("a", text="Original").get("href")
    title = soup.find("h2", class_="title").text.split(" Enhanced")[0]
    return [title, img_link]

In [21]:
mars_hemisphere_soup = BeautifulSoup(mars_hemisphere_html, "html.parser")
link_elements = mars_hemisphere_soup.findAll("a", {"class": "itemLink product-item"})
full_page_links = [dict(title=get_full_image_link("https://astrogeology.usgs.gov" + le.get('href'))[0], img_url=get_full_image_link("https://astrogeology.usgs.gov" + le.get('href'))[1]) for le in link_elements]
full_page_links

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]